# MOVIES MOVIES MOVIES !!!

In [24]:
import requests
import tarfile
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup


## Step 1: Data Collection

In [27]:
# downloading the data. These are datasets provided for non-commercial use
url= 'https://datasets.imdbws.com/title.basics.tsv.gz';
url2= 'https://datasets.imdbws.com/title.ratings.tsv.gz';

titledata= requests.get(url);
ratingdata= requests.get(url2);

title_filename= 'title_data.tsv.gz';
ratings_filename= 'ratings.tsv.gz';

# write the files to our directory
open(title_filename, 'wb').write(titledata.content);
open(ratings_filename, 'wb').write(ratingdata.content);

In [28]:
# reading the data into panda dataframes
title_df= pd.read_csv(title_filename, sep="\t", compression="gzip", dtype={'isAdult': str});
ratings_df= pd.read_csv(ratings_filename, sep="\t", compression="gzip");
print(title_df.head(5), ratings_df.head(5))

      tconst titleType            primaryTitle           originalTitle  \
0  tt0000001     short              Carmencita              Carmencita   
1  tt0000002     short  Le clown et ses chiens  Le clown et ses chiens   
2  tt0000003     short          Pauvre Pierrot          Pauvre Pierrot   
3  tt0000004     short             Un bon bock             Un bon bock   
4  tt0000005     short        Blacksmith Scene        Blacksmith Scene   

  isAdult startYear endYear runtimeMinutes                    genres  
0       0      1894      \N              1         Documentary,Short  
1       0      1892      \N              5           Animation,Short  
2       0      1892      \N              4  Animation,Comedy,Romance  
3       0      1892      \N             12           Animation,Short  
4       0      1893      \N              1              Comedy,Short         tconst  averageRating  numVotes
0  tt0000001            5.7      1971
1  tt0000002            5.8       263
2  tt0000003   

In [29]:
box_office_url = 'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/'
box_req = requests.get(box_office_url)

In [30]:
soup = BeautifulSoup(box_req.content, 'html5lib')

In [31]:
box_df= pd.read_html(io=str(soup.find('table')))[0]
box_df.head(3)

,Rank,Title,Worldwide Lifetime Gross,Domestic Lifetime Gross,Domestic %,Foreign Lifetime Gross,Foreign %,Year
0,1,Avatar,"$2,923,706,026","$785,221,649",26.9%,"$2,138,484,377",73.1%,2009
1,2,Avengers: Endgame,"$2,799,439,100","$858,373,000",30.7%,"$1,941,066,100",69.3%,2019
2,3,Avatar: The Way of Water,"$2,319,768,328","$684,018,775",29.5%,"$1,635,749,553",70.5%,2022


In [39]:
df[df['primaryTitle'] == 'Avengers: Endgame'] 

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
457307,tt10180694,tvEpisode,Avengers: Endgame,Avengers: Endgame,0,2019,\N,Talk-Show,8.6,301
460154,tt10231784,tvEpisode,Avengers: Endgame,Avengers: Endgame,0,2019,\N,"Comedy,Talk-Show",8.3,53
460568,tt10238690,tvEpisode,Avengers: Endgame,Avengers: Endgame,0,2019,\N,"Action,Adventure,Comedy",8.1,23
461226,tt10249956,tvEpisode,Avengers: Endgame,Avengers: Endgame,0,2019,\N,Comedy,8.4,30
461471,tt10255352,tvEpisode,Avengers: Endgame,Avengers: Endgame,0,2019,\N,"Comedy,Talk-Show",7.9,20
461508,tt10255990,tvEpisode,Avengers: Endgame,Avengers: Endgame,0,2019,13,Talk-Show,6.9,24
461609,tt10258872,tvEpisode,Avengers: Endgame,Avengers: Endgame,0,2019,35,"Comedy,Talk-Show",7.3,48
461637,tt10259724,tvEpisode,Avengers: Endgame,Avengers: Endgame,0,2019,\N,Comedy,8.8,7
489033,tt10761046,tvEpisode,Avengers: Endgame,Avengers: Endgame,0,2019,\N,Comedy,8.0,72
1030039,tt4154796,movie,Avengers: Endgame,Avengers: Endgame,0,2019,181,"Action,Adventure,Drama",8.4,1173968


## Step 2: Data Processing

Merge our two datasets into the same datafram

In [32]:
# when u remove and clean values
df = pd.merge(left=title_df, right=ratings_df, on='tconst', how='inner')

In [36]:
df.head(5)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
8,tt0000009,movie,Miss Jerry,Miss Jerry,False,1894,45,Romance,5.3,204
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,False,1897,100,"Documentary,News,Sport",5.3,469
326,tt0000502,movie,Bohemios,Bohemios,False,1905,100,\N,4.1,15
358,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,False,1906,70,"Action,Adventure,Biography",6.0,826
366,tt0000591,movie,The Prodigal Son,L'enfant prodigue,False,1907,90,Drama,4.4,20


In [33]:
df.drop(labels=['endYear'],axis=1, inplace=True)

Keep only the titles named movies

We need to parse certain columns to integers

In [67]:
df.dtypes

tconst             object
titleType          object
primaryTitle       object
originalTitle      object
isAdult              bool
startYear           int64
runtimeMinutes     object
genres             object
averageRating     float64
numVotes            int64
dtype: object

Here we are only looking at movies so we can drop all other forms of media (short films, shows, ...). We are analyzing these movies over time thus, we can drop any movies without a release date ('startYear'). Then, we need to convert the release dates from strings to integers so we can plot them.

In [68]:
# keep only movies
df = df[df['titleType'] == 'movie']
# drop movies without a release date
df = df[df['startYear'] != '\\N']
# convert release date from strings to integers
df['startYear'] = df['startYear'].apply(int)

Something about bias here...

In [69]:
# 1 if isAdult else, if it's any other value (0 or invalid)
# TODO comment on how this will introduce bias
function = lambda x: True if x == '1' else False
df['isAdult'] = df['isAdult'].apply(function)

In [70]:
df.head(5)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
8,tt0000009,movie,Miss Jerry,Miss Jerry,False,1894,45,Romance,5.3,204
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,False,1897,100,"Documentary,News,Sport",5.3,469
326,tt0000502,movie,Bohemios,Bohemios,False,1905,100,\N,4.1,15
358,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,False,1906,70,"Action,Adventure,Biography",6.0,826
366,tt0000591,movie,The Prodigal Son,L'enfant prodigue,False,1907,90,Drama,4.4,20


## Step 3: Exploratory Analysis and Data Vizualization

In [71]:
# make nice graphs and get some info about the data
# budget vs ratings
# box office vs ratings

## Analysis, Hypothesis Testing, and Machine Learning

In [73]:
# more in depth, test hypotheses coming from the graphs

## Insight and Policy Decision

## Conclusion